# ResOpsAUS: stations
***

***Author:** Chus Casado Rodríguez*<br>
***Date:** 12-02-2025*<br>



In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from shapely import Point
from pathlib import Path

## Configuration

In [17]:
PATH_DATASET = Path('Z:/nahaUsers/casadje/datasets/reservoirs/ResOpsAU')
PATH_OUT = PATH_DATASET / 'GIS'
PATH_OUT.mkdir(parents=True, exist_ok=True)

## Data

In [5]:
# stations reporting volume
stations_vol = pd.read_csv(PATH_DATASET / 'attributes' / 'stations_volume.csv', index_col='station_number')
stations_vol[['latitude', 'longitude']] = stations_vol[['latitude', 'longitude']].astype(float)
stations_vol['volume'] = True

print('{0} stations contain volume data'.format(len(stations_vol)))

485 stations contain volume data


In [7]:
# stations reporting level
stations_lev = pd.read_csv(PATH_DATASET / 'attributes' / 'stations_level.csv', index_col='station_number')
stations_lev[['latitude', 'longitude']] = stations_lev[['latitude', 'longitude']].astype(float)
stations_lev['level'] = True

print('{0} stations contain level data'.format(len(stations_lev)))

556 stations contain level data


## Processing

In [ ]:
# stations = stations_vol.copy()

In [8]:
# intersection and difference between the two sets of stations
ids_int = stations_vol.index.intersection(stations_lev.index)
ids_diff = stations_lev.index.difference(stations_vol.index)

In [9]:
# update 'level' attribute in 'stations_vol'
stations_vol['level'] = False
stations_vol.loc[ids_int, 'level'] = True

# update 'volume' attribute in 'stations_lev'
stations_lev['volume'] = False
stations_lev.loc[ids_int, 'volume'] = True

In [11]:
# concatenate both sets
stations = pd.concat((stations_vol, stations_lev.loc[ids_diff, :]), axis=0)
print('{0} stations contain either volume or level data'.format(len(stations)))

582 stations contain either volume or level data


In [15]:
# convert to geopandas
stations = gpd.GeoDataFrame(
    data=stations,
    geometry=[Point(xy) for xy in zip(stations.longitude, stations.latitude)],
    crs='epsg:4326'
)

In [25]:
# rename columns
stations.rename(columns={'station_number': 'number', 'station_name': 'name'}, inplace=True)
stations.index.name = 'number'

In [27]:
# export
stations.to_file(PATH_OUT / 'stations.shp')
stations.to_csv(PATH_DATASET / 'attributes' / 'stations.csv')